In [1]:
import pandas as pd
import numpy as np
import time

In [5]:
class Team:
    def __init__(self, name):
        self.name = name
        self.points = 0
        self.goals_for = 0
        self.goals_against = 0
        self.wins = 0
        self.draws = 0
        self.losses = 0
        self.played = 0
        
    def goals_scored_per_game(self):
        return round(self.goals_for/self.played,1)
    
    def goals_against_per_game(self):
        return round(self.goals_against/self.played,1)
    
    def points_per_game(self):
        return round(self.points/self.played,1)
    
    def add_result(self,result,isHome):
        [a_goals,b_goals] = [int(x) for x in result.split("-")]
        self.played += 1
        if isHome:
            self.goals_for += a_goals
            self.goals_against += b_goals
            if a_goals > b_goals:
                self.wins += 1
                self.points += 3
            elif a_goals == b_goals:
                self.draws += 1
                self.points += 1
            else:
                self.losses += 1
        else:
            self.goals_for += b_goals
            self.goals_against += a_goals
            if b_goals > a_goals:
                self.wins += 1
                self.points += 3
            elif b_goals == a_goals:
                self.draws += 1
                self.points += 1
            else:
                self.losses += 1

In [11]:
# make sure df sorted by date, date in pd.to_datetime format
# df_season['date'] = pd.to_datetime(df.date)
# df_season.sort_values(by='date',inplace=True,ascending=True)
# date is "date of next game"
def organise_teams_by_points(table_dict):
    points_dict = {}
    final_table = {}
    for team in table_dict.keys():
        points_dict[team] = table_dict[team].points
        
    position = 1
    for (key, value) in sorted(points_dict.items(), key=lambda kv: kv[1]).reverse():
        position = i
        final_table[position] = table_dict[value]
        i += 1
        
    return final_table
        

def basic_league_table(date,season,df):
    df_season = df.query("season == @season")
    mask = (df_season['date'] < date)
    df_subset = df_season.loc[mask]
    
    if len(df_subset) > 15:
        all_teams = df_subset['home'].unique().tolist()
    else:
        all_teams = set(df_subset['home'].unique.tolist() + df_subset['away'].unique.tolist())
    
    table_dict = {}

    for team in sorted(all_teams):
        t1 = Team(team)
        df_team = df_subset.query("home == @team or away == @team")
        t1.played = len(df_team)
        for i in range(len(df_team)):
            row = df_team.iloc[i]
            t1.add_result(row['score'],(row['home'] == team))
        table_dict[team] = t1
        
    final_table = organise_teams_by_points(table_dict)
    return final_table